In [1]:
# Cell 1: Imports & display options
import os
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.float_format", lambda v: f"{v:,.4f}")
pd.options.display.float_format = '{:.2f}'.format

print("✅ Imports ready.")

✅ Imports ready.


In [2]:
# Cell 2: Paths & configuration (projections + live)

# Projections (cleaned)
PROJ_INPUT  = "data/projections_L2.csv"
PROJ_OUTPUT = "../L3/data/rankings_L3.csv"

# Live (cleaned + indexed to your playerIndex)
LIVE_INPUT  = "data/live_L2.csv"
LIVE_OUTPUT = "../L3/data/rankings_LIVE_L3.csv"

# IDs expected in BOTH files
ID_COLS = ["INDEX", "PLAYER_NORM"]

# Stats used for z-scoring and ranking
STAT_COLS = ["PTS","REB","AST","STL","BLK","3PM","FG%","FT%","TO"]

# Optional context columns (included in outputs if present)
CONTEXT_COLS = ["G","MPG"]

# If any FG% / FT% happen to be in 0–100, set this True to auto-scale to 0–1
AUTO_SCALE_PCT = True

# Use weights? If True and STAT_WEIGHTS available in utils.py, we’ll weight z-scores
USE_WEIGHTS = False  # keep False to mirror “same z scoring” as requested

print("✅ Paths and config set.")

✅ Paths and config set.


In [3]:
# Cell 3: (Optional) Load STAT_WEIGHTS from utils.py if present and desired
# This cell is safe even if utils.py or STAT_WEIGHTS is missing.

STAT_WEIGHTS = None
if USE_WEIGHTS:
    try:
        import importlib.util, sys
        UTILS_PATH = "../utils/utils.py"
        spec = importlib.util.spec_from_file_location("utils_module", UTILS_PATH)
        utils_module = importlib.util.module_from_spec(spec)
        sys.modules["utils_module"] = utils_module
        spec.loader.exec_module(utils_module)

        # Prefer function get_stat_weights(), fallback to STAT_WEIGHTS dict
        get_stat_weights = getattr(utils_module, "get_stat_weights", None)
        if callable(get_stat_weights):
            STAT_WEIGHTS = get_stat_weights()
        elif hasattr(utils_module, "STAT_WEIGHTS"):
            STAT_WEIGHTS = getattr(utils_module, "STAT_WEIGHTS")

        print("✅ Weights loaded:", STAT_WEIGHTS)
    except Exception as e:
        print("⚠️ Could not load utils.py weights:", e)
else:
    print("ℹ️ USE_WEIGHTS=False — proceeding with unweighted z-score sum.")

ℹ️ USE_WEIGHTS=False — proceeding with unweighted z-score sum.


In [4]:
# Cell 4: Helper functions

def coerce_numeric(df: pd.DataFrame, cols):
    """Safely convert listed columns to numeric if they exist."""
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def maybe_scale_percentages(df: pd.DataFrame, pct_cols=("FG%", "FT%")):
    """If AUTO_SCALE_PCT=True and a pct column has values > 1, scale by 100→1."""
    if not AUTO_SCALE_PCT:
        return df
    for c in pct_cols:
        if c in df.columns:
            # Heuristic: if median > 1.5, likely 0–100 scale → convert to 0–1
            med = df[c].median(skipna=True)
            if pd.notna(med) and med > 1.5:
                df[c] = df[c] / 100.0
    return df

def compute_zscores(df: pd.DataFrame, stat_cols):
    """Compute z-scores per column; if std == 0 or NaN, assign 0.0."""
    out = df.copy()
    for c in stat_cols:
        if c not in out.columns:
            continue
        mu = out[c].mean(skipna=True)
        sd = out[c].std(ddof=0, skipna=True)  # population std for stability
        if pd.isna(sd) or sd == 0:
            out[f"{c}_z"] = 0.0
        else:
            out[f"{c}_z"] = (out[c] - mu) / sd
    return out

def build_rank(df: pd.DataFrame, stat_cols, weights=None):
    """
    Build composite RANK_SCORE:
      - Sum positive stats' z-scores (PTS, REB, AST, STL, BLK, 3PM, FG%, FT%)
      - Subtract turnovers' z-score (TO)
    If weights is provided (dict), weight the z-scores accordingly.
    """
    pos = ["PTS","REB","AST","STL","BLK","3PM","FG%","FT%"]
    neg = ["TO"]

    # Only keep stats that exist in dataframe
    pos = [c for c in pos if f"{c}_z" in df.columns]
    neg = [c for c in neg if f"{c}_z" in df.columns]

    score = pd.Series(0.0, index=df.index)

    # Helper to get weight (1.0 default when weights=None)
    def w(c):
        if weights and isinstance(weights, dict):
            return float(weights.get(c, 1.0))
        return 1.0

    for c in pos:
        score = score + df[f"{c}_z"].fillna(0.0) * w(c)
    for c in neg:
        score = score - df[f"{c}_z"].fillna(0.0) * w(c)

    out = df.copy()
    out["RANK_SCORE"] = score
    out["RANK"] = out["RANK_SCORE"].rank(method="dense", ascending=False).astype(int)
    return out

def compute_rankings(input_csv, output_csv, id_cols, stat_cols, context_cols, weights=None):
    """Full pipeline for one CSV → ranked CSV."""
    # Load
    df = pd.read_csv(input_csv)

    # Verify IDs
    missing_ids = [c for c in id_cols if c not in df.columns]
    if missing_ids:
        raise ValueError(f"Missing ID columns {missing_ids} in {input_csv}")

    # Optional percent scaling guard
    df = maybe_scale_percentages(df, pct_cols=("FG%", "FT%"))

    # Coerce numerics for stats/context
    df = coerce_numeric(df, stat_cols + context_cols)

    # Z-scores
    zdf = compute_zscores(df, stat_cols)

    # Rank (with or without weights)
    rdf = build_rank(zdf, stat_cols, weights=weights)

    # Output columns
    out_cols = (
        id_cols
        + [c for c in context_cols if c in rdf.columns]
        + stat_cols
        + ["RANK_SCORE","RANK"]
    )
    out_cols = [c for c in out_cols if c in rdf.columns]
    out = rdf[out_cols].sort_values(["RANK","RANK_SCORE"], ascending=[True, False]).reset_index(drop=True)

    # Save
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    out.to_csv(output_csv, index=False)
    print(f"✅ Saved rankings to: {output_csv}")
    display(out.head(15))
    return out

In [5]:
# Cell 5: Compute & save PROJECTIONS rankings
proj_rankings = compute_rankings(
    input_csv=PROJ_INPUT,
    output_csv=PROJ_OUTPUT,
    id_cols=ID_COLS,
    stat_cols=STAT_COLS,
    context_cols=CONTEXT_COLS,
    weights=(STAT_WEIGHTS if USE_WEIGHTS else None)
)

✅ Saved rankings to: ../L3/data/rankings_L3.csv


,INDEX,PLAYER_NORM,G,MPG,PTS,REB,AST,STL,BLK,3PM,FG%,FT%,TO,RANK_SCORE,RANK
0,10003.00,victor wembanyama,67.80,32.96,25.95,11.37,4.01,1.23,3.76,2.26,0.50,0.82,3.51,14.92,1
1,10001.00,nikola jokic,73.20,35.72,28.08,12.30,10.01,1.63,0.73,1.86,0.58,0.81,3.19,13.43,2
2,10004.00,luka doncic,70.80,35.84,30.47,8.42,8.59,1.64,0.49,3.60,0.48,0.78,3.77,11.21,3
3,10002.00,shai gilgeous alexander,74.00,33.84,32.34,5.24,6.40,1.74,0.98,1.98,0.52,0.89,2.38,11.16,4
4,10006.00,anthony davis,63.40,34.30,24.77,11.46,3.69,1.17,2.18,0.73,0.52,0.79,2.33,10.40,5
5,10005.00,giannis antetokounmpo,70.00,34.50,31.46,11.82,6.78,0.95,1.08,0.50,0.60,0.63,3.40,8.44,6
6,10021.00,derrick white,69.60,33.73,17.52,4.89,5.55,1.06,1.15,3.37,0.44,0.86,1.86,8.27,7
7,10014.00,tyrese maxey,67.40,37.82,26.09,3.51,6.09,1.46,0.41,3.16,0.44,0.87,2.18,8.11,8
8,10015.00,stephen curry,66.80,32.44,25.47,4.59,5.81,0.99,0.42,4.55,0.45,0.93,2.88,8.07,9
9,10008.00,anthony edwards,73.00,36.40,27.82,5.79,4.92,1.26,0.63,3.88,0.45,0.83,3.22,8.00,10


In [6]:
# Cell 6: Compute & save LIVE rankings
live_rankings = compute_rankings(
    input_csv=LIVE_INPUT,
    output_csv=LIVE_OUTPUT,
    id_cols=ID_COLS,
    stat_cols=STAT_COLS,
    context_cols=CONTEXT_COLS,
    weights=(STAT_WEIGHTS if USE_WEIGHTS else None)
)

✅ Saved rankings to: ../L3/data/rankings_LIVE_L3.csv


,INDEX,PLAYER_NORM,G,MPG,PTS,REB,AST,STL,BLK,3PM,FG%,FT%,TO,RANK_SCORE,RANK
0,10001.00,nikola jokic,13.00,34.50,29.20,13.40,11.10,1.80,0.80,1.80,0.65,0.86,3.20,14.61,1
1,10003.00,victor wembanyama,12.00,34.70,26.20,12.90,4.00,1.10,3.60,1.70,0.50,0.86,3.60,13.68,2
2,10004.00,luka doncic,11.00,36.80,34.60,8.50,9.00,1.90,0.50,3.50,0.47,0.79,4.10,11.11,3
3,10014.00,tyrese maxey,13.00,40.40,32.50,4.90,7.70,1.40,0.80,3.80,0.46,0.89,2.60,10.82,4
4,10002.00,shai gilgeous alexander,15.00,33.30,31.90,4.90,6.70,1.50,0.90,2.30,0.53,0.90,1.70,10.69,5
5,10036.00,kawhi leonard,6.00,33.50,24.30,5.70,3.50,2.50,0.70,2.30,0.51,0.96,1.80,9.92,6
6,10025.00,scottie barnes,14.00,32.50,19.40,7.70,5.30,1.40,1.80,1.50,0.50,0.81,1.90,9.37,7
7,10096.00,mikal bridges,13.00,35.10,16.20,4.40,4.80,2.00,1.20,2.50,0.50,0.80,1.00,9.17,8
8,10018.00,donovan mitchell,13.00,34.10,30.90,4.60,5.50,1.50,0.50,4.00,0.51,0.82,3.20,8.52,9
9,10064.00,lauri markkanen,14.00,35.90,30.60,6.10,2.10,1.00,0.50,3.70,0.48,0.89,1.40,8.18,10


In [7]:
# Cell 7: Compare by RANK_SCORE delta (LIVE vs PROJECTIONS)
# Positive RANK_SCORE_DELTA => outperforming projections
# Negative RANK_SCORE_DELTA => underperforming

compare_cols = ["INDEX", "PLAYER_NORM", "RANK", "RANK_SCORE"]

p = proj_rankings[compare_cols].rename(
    columns={"RANK": "RANK_PROJ", "RANK_SCORE": "RANK_SCORE_PROJ"}
)
l = live_rankings[compare_cols].rename(
    columns={"RANK": "RANK_LIVE", "RANK_SCORE": "RANK_SCORE_LIVE"}
)

cmp_df = p.merge(l, on=["INDEX", "PLAYER_NORM"], how="inner")

# Compute deltas
cmp_df["RANK_SCORE_DELTA"] = cmp_df["RANK_SCORE_LIVE"] - cmp_df["RANK_SCORE_PROJ"]
cmp_df["RANK_DELTA"] = cmp_df["RANK_LIVE"] - cmp_df["RANK_PROJ"]

# Sort by overperforming first
over_df = cmp_df.sort_values(["RANK_SCORE_DELTA", "RANK_LIVE"], ascending=[False, True]).reset_index(drop=True)

# Filter out players with zero projected rank score before sorting underperformers
under_df = (
    cmp_df[cmp_df["RANK_SCORE_PROJ"] != 0]
    .sort_values(["RANK_SCORE_DELTA", "RANK_LIVE"], ascending=[True, True])
    .reset_index(drop=True)
)

cols = [
    "INDEX", "PLAYER_NORM",
    "RANK_PROJ", "RANK_LIVE",
    "RANK_SCORE_PROJ", "RANK_SCORE_LIVE",
    "RANK_SCORE_DELTA", "RANK_DELTA"
]

print("✅ Positive RANK_SCORE_DELTA = outperforming projections; negative = underperforming.\n")

print("🏀 Top Overperformers (LIVE > PROJECTIONS):")
display(over_df[cols].head(20))

print("😬 Top Underperformers (LIVE < PROJECTIONS):")
display(under_df[cols].head(20))

✅ Positive RANK_SCORE_DELTA = outperforming projections; negative = underperforming.

🏀 Top Overperformers (LIVE > PROJECTIONS):


,INDEX,PLAYER_NORM,RANK_PROJ,RANK_LIVE,RANK_SCORE_PROJ,RANK_SCORE_LIVE,RANK_SCORE_DELTA,RANK_DELTA
0,10096.00,mikal bridges,120,8,3.61,9.17,5.56,-112
1,10475.00,jaylon tyson,416,76,-1.29,4.01,5.30,-340
2,10280.00,ryan kalkbrenner,217,35,1.54,6.12,4.58,-182
3,10527.00,adou thiero,540,237,-4.45,0.09,4.54,-303
4,10268.00,cedric coward,355,74,-0.34,4.11,4.45,-281
5,10410.00,drake powell,542,259,-4.70,-0.51,4.19,-283
6,10507.00,tolu smith,496,171,-2.64,1.39,4.04,-325
7,10093.00,julius randle,146,24,2.88,6.73,3.85,-122
8,10500.00,jevon carter,477,165,-2.20,1.50,3.70,-312
9,10448.00,kyle lowry,444,147,-1.65,1.99,3.64,-297


😬 Top Underperformers (LIVE < PROJECTIONS):


,INDEX,PLAYER_NORM,RANK_PROJ,RANK_LIVE,RANK_SCORE_PROJ,RANK_SCORE_LIVE,RANK_SCORE_DELTA,RANK_DELTA
0,10181.00,jared mccain,207,468,1.66,-8.06,-9.72,261
1,10304.00,thomas bryant,228,469,1.38,-8.06,-9.44,241
2,10347.00,johnny juzang,243,461,1.18,-7.54,-8.72,218
3,10133.00,robert williams,91,393,4.24,-4.00,-8.24,302
4,10259.00,cody martin,188,427,1.96,-5.25,-7.21,239
5,10318.00,larry nance,232,434,1.35,-5.46,-6.81,202
6,10233.00,guerschon yabusele,254,440,1.06,-5.72,-6.78,186
7,10239.00,justin champagnie,169,404,2.40,-4.18,-6.57,235
8,10224.00,jonathan isaac,154,390,2.62,-3.90,-6.53,236
9,10121.00,taylor hendricks,124,337,3.44,-2.75,-6.19,213
